## Enriching NCAATourneyDetailedResults.csv with Advanced Stats

#### If the opponents of a team score only 75.2 points on average, it could be more about the pace at which the team played instead of their skill on the defensive end. 

### The given Box score numbers are an incomplete standard of a team's performance. 

**Advanced Metrics ** in basketball provide a deeper understanding of a team's performance.

**Possession** is used to normalize basketball statistics - offensive/defensive efficiency and other metrics are all based on how the possession is calculated. Team performance should be measured on a per-possession basis.  

> Possession =0.96*[(Field Goal Attempts)+(Turnovers)+0.44*(Free Throw Attempts)-(Off.Rebounds)]


##### (Notice: Possession values are not calculated by using Play-By-Play data, as it seems like they do not necessarily add up to the final stats of the game and an estimation will do just fine)

For more information [click here](https://www.nbastuffer.com/analytics101/possession/)

### Now let's add some new features that can acutally be used for predictive modelling and ranking a team.


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from math import pi
import xgboost as xgb
from sklearn import *

df = pd.read_csv('../input/RegularSeasonDetailedResults.csv')

#Points Winning/Losing Team
df['WPts'] = df.apply(lambda row: 2*row.WFGM + row.WFGM3 + row.WFTM, axis=1)
df['LPts'] = df.apply(lambda row: 2*row.LFGM + row.LFGM3 + row.LFTM, axis=1)

#Calculate Winning/losing Team Possesion Feature
wPos = df.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
lPos = df.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)
#two teams use almost the same number of possessions in a game
#(plus/minus one or two - depending on how quarters end)
#so let's just take the average
df['Pos'] = (wPos+lPos)/2

### Advanced Metrics

In [ ]:
df['DayNum'].describe()

In [ ]:
#Offensive efficiency (OffRtg) = 100 x (Points / Possessions)
df['WOffRtg'] = df.apply(lambda row: 100 * (row.WPts / row.Pos), axis=1)
df['LOffRtg'] = df.apply(lambda row: 100 * (row.LPts / row.Pos), axis=1)
#Defensive efficiency (DefRtg) = 100 x (Opponent points / Opponent possessions)
df['WDefRtg'] = df.LOffRtg
df['LDefRtg'] = df.WOffRtg
#Net Rating = Off.eff - Def.eff
df['WNetRtg'] = df.apply(lambda row:(row.WOffRtg - row.WDefRtg), axis=1)
df['LNetRtg'] = df.apply(lambda row:(row.LOffRtg - row.LDefRtg), axis=1)
                         
#Assist Ratio : Percentage of team possessions that end in assists
df['WAstR'] = df.apply(lambda row: 100 * row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
df['LAstR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
#Turnover Ratio: Number of turnovers of a team per 100 possessions used.
#(TO * 100) / (FGA + (FTA * 0.44) + AST + TO
df['WTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
df['LTOR'] = df.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
                    
#The Shooting Percentage : Measure of Shooting Efficiency (FGA/FGA3, FTA)
df['WTSP'] = df.apply(lambda row: 100 * row.WPts / (2 * (row.WFGA + 0.44 * row.WFTA)), axis=1)
df['LTSP'] = df.apply(lambda row: 100 * row.LPts / (2 * (row.LFGA + 0.44 * row.LFTA)), axis=1)
#eFG% : Effective Field Goal Percentage adjusting for the fact that 3pt shots are more valuable 
df['WeFGP'] = df.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
df['LeFGP'] = df.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1)   
#FTA Rate : How good a team is at drawing fouls.
df['WFTAR'] = df.apply(lambda row: row.WFTA / row.WFGA, axis=1)
df['LFTAR'] = df.apply(lambda row: row.LFTA / row.LFGA, axis=1)
                         
#OREB% : Percentage of team offensive rebounds
df['WORP'] = df.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
df['LORP'] = df.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#DREB% : Percentage of team defensive rebounds
df['WDRP'] = df.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
df['LDRP'] = df.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)                                      
#REB% : Percentage of team total rebounds
df['WRP'] = df.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
df['LRP'] = df.apply(lambda row: (row.LDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1) 


#### PIE : Measure of a team's performance

> *A high PIE % is highly correlated to winning. In fact, a team’s PIE rating and a team’s winning percentage correlate at an R square of .908 which indicates a "strong" correlation*

from the official site of the [NBA](https://stats.nba.com/help/glossary/)

In [ ]:
df['WPIE'] = df.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
wtmp = df.apply(lambda row: row.WPts + row.WFGM + row.WFTM - row.WFGA - row.WFTA + row.WDR + 0.5*row.WOR + row.WAst +row.WStl + 0.5*row.WBlk - row.WPF - row.WTO, axis=1)
ltmp = df.apply(lambda row: row.LPts + row.LFGM + row.LFTM - row.LFGA - row.LFTA + row.LDR + 0.5*row.LOR + row.LAst +row.LStl + 0.5*row.LBlk - row.LPF - row.LTO, axis=1) 
df['WPIE'] = wtmp/(wtmp + ltmp)
df['LPIE'] = ltmp/(wtmp + ltmp)

In [ ]:

#Now let's consolidate - I've literally never read a style guide in my life
dfwins = df[['Season', 'WTeamID', 'WNetRtg', 'WPIE']]
dfwins.rename(index=str, columns={"WTeamID": "TeamID","WNetRtg": "NetRtg", "WPIE": "PIE"}, inplace = True)
dflosses = df[['Season','LTeamID', 'LNetRtg','LPIE']]
dflosses.rename(index=str, columns={"LTeamID": "TeamID","LNetRtg": "NetRtg","LPIE": "PIE"}, inplace = True)

dfadvanced = dfwins.append(dflosses)
dfadvanced = dfadvanced.groupby(['Season', 'TeamID']).agg({"NetRtg":"mean", "PIE":"mean"}).reset_index()


In [7]:
#data
games = pd.read_csv('../input/NCAATourneyCompactResults.csv')


In [9]:
games.head()

In [ ]:
#Add Ids
games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)


In [ ]:
#Add Seeds
seeds = pd.read_csv('../input/NCAATourneySeeds.csv')
seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds.values}
#add 2018 Seeds
#seeds = {**seeds, **{k.replace('1999_','1998_'):seeds[k] for k in seeds if '1999_' in k}}

games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)


In [ ]:

#Additional Features & Clean Up
games['ScoreDiff'] = games['WScore'] - games['LScore'] 
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed'] 
games = games.fillna(-1)

sdn = games.groupby(['IDTeams'], as_index=False)[['ScoreDiffNorm']].mean()


In [ ]:

#Test Set
sub = pd.read_csv('../input/SampleSubmissionStage2.csv')
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub = pd.merge(sub, sdn, how='left', on=['IDTeams'])
sub['ScoreDiffNorm'] = sub['ScoreDiffNorm'].fillna(0.)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 


In [ ]:
games.head()

In [ ]:
sub.head()

In [ ]:
games2 = games[['Season', 'ID', 'Team1', 'Team2', 'Team1Seed', 'Team2Seed', 'Pred']]
games2 = games2.merge(dfadvanced, left_on =['Season', 'Team1'], right_on = ['Season', 'TeamID'], how='inner')
games2.drop(['TeamID'], axis=1, inplace=True)
games2.rename(index=str, columns={"NetRtg": "Team1NetRtg", "PIE": "Team1PIE"}, inplace = True)
games2 = games2.merge(dfadvanced, left_on =['Season', 'Team2'], right_on = ['Season', 'TeamID'], how='inner')
games2.drop(['TeamID'], axis=1, inplace=True)
games2.rename(index=str, columns={"NetRtg": "Team2NetRtg", "PIE": "Team2PIE"}, inplace = True)


In [ ]:
df_rankings = pd.read_csv('../input/MasseyOrdinals.csv')
df_RPI = df_rankings[df_rankings['SystemName'] == 'RPI']
df_RPI_final = df_RPI[df_RPI['RankingDayNum'] == 133]
df_RPI_final.drop(labels=['RankingDayNum', 'SystemName'], inplace=True, axis=1)
df_RPI_final.head()

In [ ]:
games2 = games2.merge(df_RPI_final, left_on =['Season', 'Team1'], right_on = ['Season', 'TeamID'], how='inner')
games2.drop(['TeamID'], axis=1, inplace=True)
games2.rename(index=str, columns={"OrdinalRank": "Team1OrdinalRank"}, inplace = True)
games2 = games2.merge(df_RPI_final, left_on =['Season', 'Team2'], right_on = ['Season', 'TeamID'], how='inner')
games2.drop(['TeamID'], axis=1, inplace=True)
games2.rename(index=str, columns={"OrdinalRank": "Team2OrdinalRank"}, inplace = True)

In [ ]:
games2['Team1Seed'].replace(0, 20, inplace=True)
games2['Team2Seed'].replace(0, 20, inplace=True)
games2['SeedDiff'] = games2['Team1Seed'] - games2['Team2Seed']
games2['PIEDiff'] = games2['Team1PIE'] - games2['Team2PIE']
games2['NetRtgDiff'] = games2['Team1NetRtg'] - games2['Team2NetRtg']
games2['OrdinalRankDiff'] = games2['Team1OrdinalRank'] - games2['Team2OrdinalRank']
#games2 = games2[games2['Season']<2014]

In [ ]:
games2.head()

In [ ]:

sub2 = sub[['Season', 'ID', 'Team1', 'Team2', 'Team1Seed', 'Team2Seed', 'Pred']]
sub2['Season'] = pd.to_numeric(sub2['Season'])
sub2['Team1'] = pd.to_numeric(sub2['Team1'])
sub2['Team2'] = pd.to_numeric(sub2['Team2'])
sub2 = sub2.merge(dfadvanced, left_on =['Season', 'Team1'], right_on = ['Season', 'TeamID'], how='left')
sub2.drop(['TeamID'], axis=1, inplace=True)

In [ ]:
sub2 = sub2.merge(df_RPI_final, left_on =['Season', 'Team1'], right_on = ['Season', 'TeamID'], how='inner')
sub2.rename(index=str, columns={"OrdinalRank": "Team1OrdinalRank"}, inplace = True)
sub2.drop(['TeamID'], axis=1, inplace=True)
sub2 = sub2.merge(df_RPI_final, left_on =['Season', 'Team2'], right_on = ['Season', 'TeamID'], how='inner')
sub2.drop(['TeamID'], axis=1, inplace=True)
sub2.rename(index=str, columns={"OrdinalRank": "Team2OrdinalRank"}, inplace = True)

In [ ]:
sub2.rename(index=str, columns={"NetRtg": "Team1NetRtg", "PIE": "Team1PIE"}, inplace = True)
sub2 = sub2.merge(dfadvanced, left_on =['Season', 'Team2'], right_on = ['Season', 'TeamID'], how='left')
sub2.drop(['TeamID'], axis=1, inplace=True)
sub2.rename(index=str, columns={"NetRtg": "Team2NetRtg", "PIE": "Team2PIE"}, inplace = True)
sub2['Team1Seed'].replace(0, 20, inplace=True)
sub2['Team2Seed'].replace(0, 20, inplace=True)


In [ ]:
sub2['SeedDiff'] = sub2['Team1Seed'] - sub2['Team2Seed']
sub2['PIEDiff'] = sub2['Team1PIE'] - sub2['Team2PIE']
sub2['NetRtgDiff'] = sub2['Team1NetRtg'] - sub2['Team2NetRtg']
sub2['OrdinalRankDiff'] = sub2['Team1OrdinalRank'] - sub2['Team2OrdinalRank']

In [ ]:
games2.tail()

In [ ]:
# TrueSkill is a rating system based on Bayesian inference, estimating each players skill as a gaussian like Elo rating.
# See trueskill.org for more.

#import pandas as pd, numpy as np
from trueskill import TrueSkill, Rating, rate_1vs1

ts = TrueSkill(draw_probability=0.01) # 0.01 is arbitary small number
beta = 25 / 6  # default value

def win_probability(p1, p2):
    delta_mu = p1.mu - p2.mu
    sum_sigma = p1.sigma * p1.sigma + p2.sigma * p2.sigma
    denom = np.sqrt(2 * (beta * beta) + sum_sigma)
    return ts.cdf(delta_mu / denom)
    
submit = pd.read_csv('../input/SampleSubmissionStage2.csv')
submit[['Season', 'Team1', 'Team2']] = submit.apply(lambda r:pd.Series([int(t) for t in r.ID.split('_')]), axis=1)

df_tour = pd.read_csv('../input/RegularSeasonCompactResults.csv')
teamIds = np.unique(np.concatenate([df_tour.WTeamID.values, df_tour.LTeamID.values]))
ratings = { tid:ts.Rating() for tid in teamIds }

def feed_season_results(season):
    print("season = {}".format(season))
    df1 = df_tour[df_tour.Season == season]
    for r in df1.itertuples():
        ratings[r.WTeamID], ratings[r.LTeamID] = rate_1vs1(ratings[r.WTeamID], ratings[r.LTeamID])

def update_pred(season):
    beta = np.std([r.mu for r in ratings.values()]) 
#    print("beta = {}".format(beta))
    games2.loc[games2.Season==season, 'Pred2'] = games2[games2.Season==season].apply(lambda r:win_probability(ratings[r.Team1], ratings[r.Team2]), axis=1)
    sub2.loc[sub2.Season==season, 'Pred2'] = sub2[sub2.Season==season].apply(lambda r:win_probability(ratings[r.Team1], ratings[r.Team2]), axis=1)

for season in sorted(df_tour.Season.unique()): # exclude last 4 years
    feed_season_results(season)

update_pred(1985)
update_pred(1986)
update_pred(1987)
update_pred(1988)
update_pred(1989)
update_pred(1999)
update_pred(2000)
update_pred(2001)
update_pred(2002)
update_pred(2003)
update_pred(2004)
update_pred(2005)
update_pred(2006)
update_pred(2007)
update_pred(2008)
update_pred(2009)
update_pred(2010)
update_pred(2011)
update_pred(2012)
update_pred(2013)
update_pred(2014)
feed_season_results(2014)
update_pred(2015)
feed_season_results(2015)
update_pred(2016)
feed_season_results(2016)
update_pred(2017)

#submit.drop(['Season', 'Team1', 'Team2'], axis=1, inplace=True)
#submit.to_csv('trueskill_estimation.csv', index=None)

In [ ]:
sub2['Pred'].describe()

In [ ]:
sub2.head()

In [ ]:
col = [c for c in games2.columns if c not in ['ID', 'Season', 'Team1', 'Team2','IDTeams','IDTeam1','IDTeam2','Pred','DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'ScoreDiff', 'Team1NetRtg','Team2NetRtg','Team1Seed','Team2Seed','Team1PIE','Team2PIE','Team1OrdinalRank', 'Team2OrdinalRank']]
#col = [c for c in games2.columns if c not in ['ID', 'Season', 'Team1', 'Team2','IDTeams','IDTeam1','IDTeam2','Pred','DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'ScoreDiff', 'Team1NetRtg','Team2NetRtg','Team1Seed','Team2Seed','Team1PIE','Team2PIE','Team1OrdinalRank', 'Team2OrdinalRank']]

In [ ]:
xgbmodel = xgb.XGBRegressor(gamma=1)
xgbmodel.fit(games2[col],games2['Pred'], eval_metric='logloss')
sub2['Pred'] = xgbmodel.predict(sub2[col],output_margin=False)
#sub2['Pred'] = (sub2['Pred3']+sub2['Pred2'])/2
sub2[['ID','Pred']].to_csv('rh3p_submission.csv', index=False)

In [ ]:
xgb.to_graphviz(model)